In [ ]:
# Sesuaikan config berikut
DATA_DIR    = "/workspace/SPLIT_RHYTHM_NPY/train"   # ganti dengan path rhythm
MODEL_BASE  = "/workspace/HASIL/DECODER_RHYTHM/HASIL_1"
LABEL_MAP   = {'N':0, 'AFIB':1, 'VT':2, 'VFL':3}     # sesuaikan dengan kelas rhythm mu
MAX_LEN     = 1800                                  # sesuaikan durasi dalam sample count

# Sisanya kode **PERSIS** sama dengan beats-mu yang barusan, hanya 3 bagian di atas saja yang diubah.


In [ ]:
import os, json, numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt, seaborn as sns, shutil
from tqdm import tqdm

# CONFIG
DATA_DIR = "/workspace/SPLIT_SLIDING_FINAL/train"
OUTPUT_DIR = "/workspace/HASIL_DECODER_KFOLD/Hasil_Tuning"
LABEL_MAP = {'N': 0, 'AFIB': 1, 'VFL': 2}
N_SPLITS = 5
SEED = 42
EPOCHS = 20
BATCH_SIZE = 32
LR = 2e-5
MAX_LEN = 512
D_MODEL = 768
N_HEAD = 12
N_LAYERS = 12

torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DATA LOADING
... # sama seperti kode Anda sebelumnya

# MODEL
... # sama seperti SimpleDecoderOnly sebelumnya

# METRICS
... # sama seperti specificity_per_class

# TRAINING
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
all_results, best_f1, best_fold = [], 0, 0

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n[INFO] Fold {fold}")
    train_set = ECGDataset([X[i] for i in train_idx], y[train_idx])
    val_set = ECGDataset([X[i] for i in val_idx], y[val_idx])
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE)

    model = SimpleDecoderOnly(num_classes=len(LABEL_MAP)).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    epoch_history = []
    for epoch in range(1, EPOCHS+1):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for xb, yb in tqdm(train_loader, desc=f"Fold {fold} Epoch {epoch}"):
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * xb.size(0)
            correct += (out.argmax(1) == yb).sum().item()
            total += yb.size(0)
        train_loss = total_loss / total
        train_acc = correct / total

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                val_loss += loss.item() * xb.size(0)
                correct_val += (out.argmax(1) == yb).sum().item()
                total_val += yb.size(0)
        val_loss /= total_val
        val_acc = correct_val / total_val

        print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        epoch_history.append({'epoch': epoch, 'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': val_loss, 'val_acc': val_acc})

    # Evaluation, saving models, history, confusion matrix, etc (sama seperti sebelumnya)

final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv(os.path.join(OUTPUT_DIR, "hasil_kfold_decoder.csv"), index=False)
print(f"[INFO] Fold terbaik berdasarkan F1-score: Fold {best_fold}")


[INFO] Fold 1


Fold 1 Epoch 1: 100%|██████████| 120/120 [00:06<00:00, 17.57it/s]


[Epoch 1] Train Loss: 0.5470, Train Acc: 0.7698, Val Loss: 0.2390, Val Acc: 0.9427


Fold 1 Epoch 2: 100%|██████████| 120/120 [00:06<00:00, 17.29it/s]


[Epoch 2] Train Loss: 0.2240, Train Acc: 0.9328, Val Loss: 0.2171, Val Acc: 0.9437


...

Fold 5 Epoch 19: 100%|██████████| 120/120 [00:06<00:00, 17.30it/s]


[Epoch 19] Train Loss: 0.0844, Train Acc: 0.9703, Val Loss: 0.1261, Val Acc: 0.9688


Fold 5 Epoch 20: 100%|██████████| 120/120 [00:06<00:00, 18.26it/s]


[Epoch 20] Train Loss: 0.0741, Train Acc: 0.9737, Val Loss: 0.1387, Val Acc: 0.9667


ValueError: No objects to concatenate